In [ ]:
import tensorflow as tf
import keras

from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import Adam
import pickle
import numpy as np
import os

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
file = open("metamorphosis_clean.txt", "r", encoding = "utf8")

# store file in list
lines = []
for i in file:
    lines.append(i)

# Convert list to string
data = ""
for i in lines:
  data = ' '. join(lines)

#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces
data = data.split()
data = ' '.join(data)
data[:500]
len(data)

118461

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]
len(sequence_data)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)

print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])

X = np.array(X)
y = np.array(y)
print("Data: ", X[:10])
print("Response: ", y[:10])
y = to_categorical(y, num_classes=vocab_size)
y[:5]

2617
The Length of sequences are:  22044
Data:  [[  52  139   55]
 [ 139   55   14]
 [  55   14   93]
 [  14   93  935]
 [  93  935   28]
 [ 935   28 1313]
 [  28 1313  936]
 [1313  936    4]
 [ 936    4  241]
 [   4  241   42]]
Response:  [  14   93  935   28 1313  936    4  241   42 1314]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 10)             26170     
                                                                 
 lstm (LSTM)                 (None, 1, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 2617)              2619617   
                                                                 
Total params: 15694787 (59.87 MB)
Trainable params: 15694787 (59.87 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=150, batch_size=64, callbacks=[checkpoint])

Epoch 1/150
345/345 [==============================] - ETA: 0s - loss: 6.2995
Epoch 1: loss improved from inf to 6.29954, saving model to next_words.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


345/345 [==============================] - 15s 28ms/step - loss: 6.2995
Epoch 2/150
345/345 [==============================] - ETA: 0s - loss: 5.9030
Epoch 2: loss improved from 6.29954 to 5.90300, saving model to next_words.h5
345/345 [==============================] - 6s 16ms/step - loss: 5.9030
Epoch 3/150
345/345 [==============================] - ETA: 0s - loss: 5.6400
Epoch 3: loss improved from 5.90300 to 5.64004, saving model to next_words.h5
345/345 [==============================] - 6s 16ms/step - loss: 5.6400
Epoch 4/150
345/345 [==============================] - ETA: 0s - loss: 5.3771
Epoch 4: loss improved from 5.64004 to 5.37711, saving model to next_words.h5
345/345 [==============================] - 6s 17ms/step - loss: 5.3771
Epoch 5/150
345/345 [==============================] - ETA: 0s - loss: 5.1044
Epoch 5: loss improved from 5.37711 to 5.10442, saving model to next_words.h5
345/345 [==============================] - 5s 15ms/step - loss: 5.1044
Epoch 6/150
345/345 

In [ ]:
from keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""

  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break

  print(predicted_word)
  return predicted_word
while(True):
  text = input("Enter your line: ")

  if text == "0":
      print("Execution completed.....")
      break

  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)

          Predict_Next_Words(model, tokenizer, text)

      except Exception as e:
        print("Error occurred: ",e)
        continue

Enter your line: he had never
['he', 'had', 'never']
1/1 [==============================] - 1s 807ms/step
felt
Enter your line: it was a
['it', 'was', 'a']
1/1 [==============================] - 0s 21ms/step
strain
Enter your line: 0
Execution completed.....
